In [13]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split, Subset
from torch import Tensor

from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split

import numpy as np
import matplotlib.pyplot as plt

import os

print(os.getcwd()) # dosya yolunu ver
%run ../Model.ipynb
%run ../Dataset.ipynb

%run ./Flow_Model.ipynb

plt.rcParams['font.size'] = 14
plt.rcParams['font.family'] = 'DeJavu Serif'
plt.rcParams['font.serif'] = ['Times New Roman']

/arf/home/tunal/ondemand/PhD Thesis Starting/01_SON/Tik-4/Tez/05-MNIST/07-UAE_Latent_FM


In [14]:
# Hyper-Parameters & Settings
batch_size = 500

In [15]:
# Dataset
train_dataset = MNISTDataset(mode='train')
val_dataset = MNISTDataset(mode='val')
test_dataset = MNISTDataset(mode='test')

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=True)

In [16]:
# UAE modelini çalıştır

path = '../02-UAE_for_MNIST/results/UAE_MNIST'
model = torch.load(path + '.model', weights_only=False)
model.eval()

To_Uniform(
  (encoder): Sequential(
    (0): Linear(in_features=784, out_features=2000, bias=True)
    (1): SiLU()
    (2): BatchNorm1d(2000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Linear(in_features=2000, out_features=2000, bias=True)
    (4): SiLU()
    (5): BatchNorm1d(2000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Linear(in_features=2000, out_features=2000, bias=True)
    (7): SiLU()
    (8): BatchNorm1d(2000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Linear(in_features=2000, out_features=2000, bias=True)
    (10): SiLU()
    (11): BatchNorm1d(2000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): Linear(in_features=2000, out_features=3, bias=True)
    (13): Sigmoid()
  )
  (decoder): Sequential(
    (0): Linear(in_features=3, out_features=2000, bias=True)
    (1): SiLU()
    (2): Linear(in_features=2000, out_features=2000, bias=True)
    (3): SiLU()
    (4): L

In [17]:
# "results" klasörünü oluştur (zaten varsa hata vermez)
os.makedirs("results", exist_ok=True)

flow_name = './results/UAE_Latent_FM'
flow = Flow()

optimizer = torch.optim.Adam(flow.parameters(), 0.0003)
loss_fn = nn.MSELoss()

In [18]:
train_cost = []
val_cost = []
best_val_loss = float('inf')  # Başlangıçta çok büyük bir değer

for epoch in range(1000):
    epoch_losses = []

    flow.train()
    for x, _ in train_loader:

        x_1 = model.encoder(x)  # encode edilmiş temsil (batch_size x latent_dim)
        
        x_0 = torch.rand_like(x_1)
        t = torch.rand(len(x_1), 1)

        x_t = (1 - t) * x_0 + t * x_1
        dx_t = x_1 - x_0

        optimizer.zero_grad()
        pred = flow(t=t, x_t=x_t)
        loss = loss_fn(pred, dx_t)
        loss.backward()
        optimizer.step()

        epoch_losses.append(loss.item())

    mean_train_loss = sum(epoch_losses) / len(epoch_losses)
    train_cost.append(mean_train_loss)

    # Validation
    flow.eval()
    val_losses = []
    with torch.no_grad():
        for x_val, _ in val_loader:
            x_1_val = model.encoder(x_val) # encode edilmiş temsil (batch_size x latent_dim)
            
            x_0_val = torch.rand_like(x_1_val)
            t_val = torch.rand(len(x_1_val), 1)

            x_t_val = (1 - t_val) * x_0_val + t_val * x_1_val
            dx_t_val = x_1_val - x_0_val

            pred_val = flow(t=t_val, x_t=x_t_val)
            val_loss = loss_fn(pred_val, dx_t_val)
            val_losses.append(val_loss.item())

    mean_val_loss = sum(val_losses) / len(val_losses)
    val_cost.append(mean_val_loss)

    # En iyi modeli val loss'a göre kaydet
    if mean_val_loss < best_val_loss:
        print('Saved!')
        best_val_loss = mean_val_loss
        torch.save(flow, flow_name + '.model')

    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch+1}/1000 - Train Loss: {mean_train_loss:.4f} - Val Loss: {mean_val_loss:.4f}")

# Son epoch modelini ayrı istersen:
torch.save(flow, flow_name + '_final.model')

Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Epoch 100/1000 - Train Loss: 0.1142 - Val Loss: 0.1129
Saved!
Saved!
Epoch 200/1000 - Train Loss: 0.1137 - Val Loss: 0.1126
Epoch 300/1000 - Train Loss: 0.1141 - Val Loss: 0.1121
Saved!
Epoch 400/1000 - Train Loss: 0.1136 - Val Loss: 0.1135
Epoch 500/1000 - Train Loss: 0.1141 - Val Loss: 0.1123
Epoch 600/1000 - Train Loss: 0.1138 - Val Loss: 0.1115
Saved!
Epoch 700/1000 - Train Loss: 0.1138 - Val Loss: 0.1109
Saved!
Epoch 800/1000 - Train Loss: 0.1131 - Val Loss: 0.1126
Saved!
Epoch 900/1000 - Train Loss: 0.1134 - Val Loss: 0.1112
Epoch 1000/1000 - Train Loss: 0.1133 - Val Loss: 0.1129


In [19]:
# CSV dosyasına kaydet
train_losses = train_cost  # liste veya numpy array
val_losses = val_cost

np.savetxt("results/losses.csv", 
           np.column_stack((train_losses, val_losses)), 
           delimiter=",", 
           header="train_loss,val_loss", 
           comments="")